In [1]:
import psycopg2

import pandas as pd
import numpy as np

from copy import deepcopy

import ast
import random
import networkx as nx
import time, unicodedata
import itertools

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed
from datetime import datetime

In [3]:
twitter_users = pd.read_csv("../REST/static/filtered_twitter_users.csv",index_col="id")
twitter_users.sample(5)

,followers_count,friends_count,is_org,lang,match_name,match_ratio,name,screen_name,truncated_id,community
id,,,,,,,,,,
2.232249e+09,66.0,172.0,False,en,ibrahim kurban,97.0,İbrahim Kurban,ikrbn6465,223224940,1
5.893993e+08,34.0,29.0,False,en,alper kodak,100.0,Alper Kodak,kodakalper1,589399262,1
1.468747e+08,276.0,160.0,False,tr,halime karayel,100.0,Halime Karayel,Haluket,146874707,1
6.664567e+07,79.0,161.0,False,tr,kutsal koruk,100.0,Kutsal Koruk,kutsalkoruk,66645671,1
7.565966e+17,48.0,221.0,False,tr,sena gulseren,96.0,Sena Gülseren,senagulseren_,756596649,1


In [2]:
cons = pd.read_csv("../REST/static/filtered_twitter_connections.csv",index_col="id")
cons.sample(5, random_state=42)

,from_user_id,to_user_id,formation,first_date
id,,,,
4445,4354816696,113954902,{'2018.05.24': True},True
4118,3299537073,106086098,{'2018.05.08': True},True
3893,2940722500,955847747093057537,{'2018.05.08': True},True
4210,3407552944,334139315,{'2018.05.24': True},True
5603,945405496466575360,847741204204634112,{'2018.05.08': True},True


In [3]:
import ast
str2dict = lambda d : ast.literal_eval(d)
truncate = lambda x: int(str(int(x))[:9])

In [4]:
cons.formation = cons.formation.apply(str2dict)

In [5]:
def present_in_date(changes_dates, queried_date):
    """
    checking if a connection is present in a queried date
    changes_dates: {d1:True, d2:False, d3:True} connection added or removed
    queried_date: e.g. "2018.05.08"
    """
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08
    changes = sorted(changes_dates,key=lambda d: str2date(d))
    queried_date = datetime.strptime(queried_date, '%Y.%m.%d')
    present = False
    for d in changes:
        if queried_date < str2date(d):
            break
        present = changes_dates[d]
    return present

In [6]:
def get_dates(cons):
    all_dates = set()
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08

    for dates in cons.formation.apply(lambda x: list(x)):
        for date in dates:
            all_dates.add(str2date(date))
    return [d.strftime('%Y.%m.%d') for d in sorted(all_dates)]

In [12]:
dates = get_dates(cons)
d1 = dates[0]
d2 = dates[-1]

In [13]:
def get_connections_by_date(cons, date, present=True):
    nw = deepcopy(cons)
    for_col = nw.formation.apply(lambda dates: present_in_date(dates,date))
    return cons[for_col == present]

In [14]:
def calculate_new_edges(cons, d1, d2):
    return get_connections_by_date(get_connections_by_date(cons, d2), d1, False)

In [15]:
s = time.time()
new_edges = calculate_new_edges(cons, d1, d2)
print("took: ",time.time()-s)

took:  0.2541506290435791


In [17]:
twitter_users = pd.read_csv("../REST/static/filtered_twitter_users.csv", index_col="id")
twitter_users.sample(5)

,match_name,screen_name,match_ratio,name,lang,followers_count,friends_count,truncated_id,is_org
id,,,,,,,,,
1.289458e+09,suha demirsoy,suha_demirsoy,96,süha demirsoy,tr,18.0,36.0,128945774,False
1.685805e+08,erkan akdemir,srkn_akdemir,96,serkan akdemir,en,69.0,144.0,168580477,False
9.442261e+08,sumeyra yerinde,sumeyrayerinde,97,Sümeyra Yerinde,tr,110.0,94.0,944226110,False
3.341393e+08,beyza cengiz,Cengizbeyza,100,Beyza Cengiz,tr,642.0,314.0,334139315,False
1.406830e+07,emrah kaya,emrahkaya,100,Emrah Kaya,en,87334.0,438.0,14068305,False


### Categorizing edges (user-user vs user-foci)

In [23]:
orgs_df = pd.read_csv("../REST/static/orgs.csv", index_col="id")
orgs_df.sample(5)

,name,screen_name,lang,match_name,match_ratio,followers_count,friends_count,truncated_id,is_org
id,,,,,,,,,
7.819232e+17,Şehir Gerçekleri,sehirgercekleri,tr,Şehir Üniversitesi (İnsan ve Toplum Bilimleri...,86.0,9.0,120.0,781923229,True
2.361548e+09,cefa,cefasehirfm,tr,Dspace Admin,68.0,235.0,463.0,236154753,True
2.847234e+09,Şehir Muzik Kulübü,sehirmusicclub,tr,Şehir Üniversitesi (İnsan ve Toplum Bilimleri...,86.0,243.0,250.0,284723384,True
8.057471e+17,In-House @ Şehir,inhouse_sehir,tr,Şehir Üniversitesi (İnsan ve Toplum Bilimleri...,86.0,41.0,8.0,805747110,True
5.330558e+08,Şehirli Aslanlar,uA_SehirUNI,tr,! Sehir Sanat,69.0,1672.0,61.0,533055810,True


In [24]:
orgs = set(orgs_df.index)

In [25]:
is_focal = lambda from_,to, orgs: (0,0) if (from_ in orgs and to in orgs)\
                                        or not(from_ in orgs or to in orgs)\
                                            else ((to,from_) if to in orgs else (from_,to))

In [27]:
new_edges["foci"] = new_edges.apply(lambda row:is_focal(row['from_user_id'], row['to_user_id'], orgs), axis=1)
new_edges["nonfoci"] = new_edges.foci.apply(lambda f: f[1])
new_edges["foci"] = new_edges.foci.apply(lambda f: f[0])
new_edges[new_edges.foci!=0].sample(5)

,from_user_id,to_user_id,formation,first_date,foci,nonfoci
id,,,,,,
69,47148388,1945992630,{'2018.05.24': True},True,1945992630,47148388
4388,4217413882,618131962,{'2018.05.24': True},True,4217413882,618131962
4242,3627550275,2612870692,{'2018.05.24': True},True,3627550275,2612870692
5725,995255575398240256,1666891914,{'2018.05.24': True},True,1666891914,995255575398240256
5120,831060584166719489,106086098,{'2018.05.24': True},True,106086098,831060584166719489


# Calculating membership closures
### if a user u1 just got a new connection with foci f1 at time t2, get friends of u1 who had connections with f1 at time t1
<img src="../misc/member.png">

In [54]:
t1 = get_connections_by_date(cons, d1)
t1["foci"] = t1.apply(lambda row:is_focal(row['from_user_id'], row['to_user_id'], orgs), axis=1)
t1["nonfoci"] = t1.foci.apply(lambda f: f[1])
t1["foci"] = t1.foci.apply(lambda f: f[0])
t1[t1.foci!=0].sample(5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,from_user_id,to_user_id,formation,first_date,foci,nonfoci
id,,,,,,
1120,374146195,2940722500,{'2018.05.08': True},True,2940722500,374146195
2516,1439588580,106086098,{'2018.05.08': True},True,106086098,1439588580
1711,589516467,4415560576,{'2018.05.08': True},True,4415560576,589516467
4021,3093872810,174415744,{'2018.05.08': True},True,174415744,3093872810
2078,802893386,737056442,{'2018.05.08': True},True,737056442,802893386


In [55]:
# foci'es present at t1 and t2
mutual_foci = new_edges[new_edges.foci!=0].merge(t1[t1.foci!=0], on="foci")
mutual_foci.sample(5)

,from_user_id_x,to_user_id_x,formation_x,first_date_x,foci,nonfoci_x,from_user_id_y,to_user_id_y,formation_y,first_date_y,nonfoci_y
4569,995255575398240256,1222491402,{'2018.05.24': True},True,1222491402,995255575398240256,3234065324,1222491402,{'2018.05.08': True},True,3234065324
4596,995255575398240256,3064906390,{'2018.05.24': True},True,3064906390,995255575398240256,153019444,3064906390,{'2018.05.08': True},True,153019444
3574,996760635730952194,106086098,{'2018.05.24': True},True,106086098,996760635730952194,106086098,162310009,{'2018.05.08': True},True,162310009
4281,831060584166719489,609301446,{'2018.05.24': True},True,609301446,831060584166719489,886280869991895041,609301446,{'2018.05.08': True},True,886280869991895041
4488,995255575398240256,1222491402,{'2018.05.24': True},True,1222491402,995255575398240256,465062899,1222491402,{'2018.05.08': True},True,465062899


## check for membership closure

In [56]:
def membership_closure(cons, u1, u2):
    truncate = lambda x: int(str(int(x))[:9])
    u1_friends = set(
        cons.apply(lambda row: truncate(row['to_user_id'])if truncate(u1) == truncate(row['from_user_id'])\
                 else (truncate(row['from_user_id']) if truncate(u1) == truncate(row['to_user_id'])\
                   else np.NaN), axis=1).dropna())
    return u2 in u1_friends

In [57]:
start = time.time()
mutual_foci['membership_closure'] = mutual_foci.apply(lambda row: 
                                                      membership_closure(t1,
                                                                         row['nonfoci_x'],
                                                                         row['nonfoci_y']),axis=1)
print("Took:", time.time()-start)
print("There are", sum(mutual_foci["membership_closure"]), "membership_closures")
mutual_foci.sample(5)

Took: 395.7770209312439
There are 0 membership_closures


,from_user_id_x,to_user_id_x,formation_x,first_date_x,foci,nonfoci_x,from_user_id_y,to_user_id_y,formation_y,first_date_y,nonfoci_y,membership_closure
2702,932632343160020993,106086098,{'2018.05.24': True},True,106086098,932632343160020993,3298803633,106086098,{'2018.05.08': True},True,3298803633,False
2674,932632343160020993,106086098,{'2018.05.24': True},True,106086098,932632343160020993,2948587163,106086098,{'2018.05.08': True},True,2948587163,False
4478,995255575398240256,1222491402,{'2018.05.24': True},True,1222491402,995255575398240256,342114048,1222491402,{'2018.05.08': True},True,342114048,False
4376,995255575398240256,737056442,{'2018.05.24': True},True,737056442,995255575398240256,512080374,737056442,{'2018.05.08': True},True,512080374,False
2616,932632343160020993,106086098,{'2018.05.24': True},True,106086098,932632343160020993,2336600952,106086098,{'2018.05.08': True},True,2336600952,False


# Calculating focal closures
### if a user u1 just got a new connection with another user u2 at time t2, and u1 and u2 have been connected to foci f1 at time t1
<img src="../misc/focal.png">

In [39]:
# Get all new user-user connections at t2
new_u2u_edges = new_edges[new_edges.foci==0]
new_u2u_edges.sample(5)

,from_user_id,to_user_id,formation,first_date,foci,nonfoci
id,,,,,,
2416,1241036822,618131962,{'2018.05.24': True},True,0,0
704,233548506,618131962,{'2018.05.24': True},True,0,0
3632,2810110713,618131962,{'2018.05.24': True},True,0,0
1054,353631855,3421922319,{'2018.05.24': True},True,0,0
3093,2255825730,1537318436,{'2018.05.24': True},True,0,0


In [44]:
foci_t1 = t1[t1.foci!=0]
foci_t1.sample(5)

,from_user_id,to_user_id,formation,first_date,foci,nonfoci
id,,,,,,
226,90976741,2847233849,{'2018.05.08': True},True,2847233849,90976741
2304,1076267664,2940722500,{'2018.05.08': True},True,2940722500,1076267664
1090,365164239,2940722500,{'2018.05.08': True},True,2940722500,365164239
1214,411064560,4091594428,{'2018.05.08': True},True,4091594428,411064560
4830,783600969393303556,2940722500,{'2018.05.08': True},True,2940722500,783600969393303556


In [46]:
users_with_mut_foci = foci_t1.groupby("foci")['nonfoci'].apply(lambda x: set(x)).reset_index()
users_with_mut_foci = users_with_mut_foci[users_with_mut_foci.nonfoci.apply(lambda x: len(x)>1)]
users_with_mut_foci.sample(5)

,foci,nonfoci
18,2529427087,"{2866935552, 709465725648424960, 7688711934704..."
8,737056442,"{764021907171663872, 798186610583928833, 79882..."
3,443551180,"{723926465679646720, 745389817765203968, 76023..."
15,2207630876,"{342114048, 916911351632711680, 70946572564842..."
24,3892757176,"{989792178183471104, 704764994857078784, 70946..."


In [47]:
def is_fclosure(f, t, users_with_mut_foci):
    for _, focal in users_with_mut_foci.iterrows():
        if f in focal["nonfoci"] and t in focal["nonfoci"]:
            return True
    return False

In [48]:
new_u2u_edges.sample(5)

,from_user_id,to_user_id,formation,first_date,foci,nonfoci
id,,,,,,
20,32660008,253616057,{'2018.05.24': True},True,0,0
3,14668733,1537318436,{'2018.05.24': True},True,0,0
3902,2966940226,113954902,{'2018.05.24': True},True,0,0
5497,907489214643818496,618131962,{'2018.05.24': True},True,0,0
4999,800742744838709248,162310009,{'2018.05.24': True},True,0,0


In [53]:
start = time.time()
new_u2u_edges["is_focal_closure"] = new_u2u_edges.apply(
    lambda row: is_fclosure(
        row["from_user_id"], row["to_user_id"], users_with_mut_foci), axis=1)
print("took:", time.time()-start)
print("There are",sum(new_u2u_edges["is_focal_closure"]), "focal closures")
new_u2u_edges.sample(5)

took: 0.5408101081848145
There are 638 focal closures


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,from_user_id,to_user_id,formation,first_date,foci,nonfoci,is_focal_closure
id,,,,,,,
4125,3311816145,32660008,{'2018.05.24': True},True,0,0,True
2627,1537318436,1439588580,{'2018.05.24': True},True,0,0,True
2635,1537318436,2301123770,{'2018.05.24': True},True,0,0,True
4435,4306171395,618131962,{'2018.05.24': True},True,0,0,False
24,32660008,791665500,{'2018.05.24': True},True,0,0,True
